# Método de iteración funcional para sistemas de ecuaciones

Entradas:

 - una función $g: \mathbb{R}^N \to \mathbb{R}^N$,
 - $x_0 \in \mathbb{R}^N$.
 
y, según la variante:
 
  1. tolerancia entre dos iteraciones consecutivas,
  2. solución y tolerancia,
  3. constante de Lipstchiz y tolerancia.

Primero, definimos un generador infinito de los términos de la sucesión.

In [1]:
def functional_iteration(g, x_0):
    x = x_0
    while True:
        x = g(*x)
        yield x

In [19]:
def newton(g, x_0):
    x = x_0
    f = g.derivative()
    while True:
        y = f(*x).solve_right(g(*x))
        x = vector(x) - vector(y)
        yield x

In [3]:
import itertools as it

g(x,y) = (x*y,x+y)
gen = functional_iteration(g, (1,2))

for i in it.islice(gen,1,10):
    print(i)

(6, 5)
(30, 11)
(330, 41)
(13530, 371)
(5019630, 13901)
(69777876630, 5033531)
(351229105131280530, 69782910161)
(24509789089304573335878465330, 351229174914190691)
(8608552999157278550998626549630446732052243030, 24509789089655802510792656021)


Para la primera variante, creamos un generador, a partir del primero, que nos dé la distancia entre dos aproximaciones consecutivas, y nos quedamos con el primer elemento del generador para el que la distancia sea menor que la tolerancia dada.

In [43]:
def adjacent_distance(method, g, x_0):
    gen = method(g, x_0)
    x = gen.next()
    
    for i in gen:
        yield (i,abs(x-i))
        x = i
        

In [44]:
for i in it.islice(adjacent_distance(functional_iteration, g, (1,2)),1,10):
    print(i)

((30, 11), 6*sqrt(17))
((330, 41), 30*sqrt(101))
((13530, 371), 330*sqrt(1601))
((5019630, 13901), 13530*sqrt(136901))
((69777876630, 5033531), 5019630*sqrt(193210001))
((351229105131280530, 69782910161), 69777876630*sqrt(25336424260901))
((24509789089304573335878465330, 351229174914190691), 69777876630*sqrt(123379682716031369741064342213329561))
((8608552999157278550998626549630446732052243030, 24509789089655802510792656021), 69777876630*sqrt(15220356191571673096601625389721419779642653785332869973603759289761181))
((210993818376468802695338859585511727362673276231266487027284989424284783630, 8608552999157278575508415639286249242844899051), 9699124851570*sqrt(473232283041107039915980536921443272533265969435656341948192334832818735256916412888952020860478229687573371265211081161241))


In [45]:
def first_variant(method, g, x_0, epsilon):
    orig_gen = adjacent_distance(method,g,x_0)
    gen = it.takewhile(lambda (x,e) : e >= epsilon, orig_gen)
    
    return it.chain(gen, it.islice(orig_gen,1))

In [46]:
def second_variant(method, g, x_0, sol, epsilon):
    orig_gen = method(g,x_0)
    gen = it.takewhile(lambda (x) : abs(x-sol) >= epsilon, orig_gen)
    
    return it.chain(gen, it.islice(orig_gen,1))

In [47]:
def third_variant(g, x_0, L, epsilon):
    n_iterations = int((log(epsilon*(1-L)/abs(vector(g(*x_0))-vector(x_0)))/log(L)))+2
    
    print("Iterations: {}".format(n_iterations))
    
    return it.islice(functional_iteration(g,x_0), n_iterations)

In [48]:
def print_all(gen):
    for i in gen:
        print(i)
        
def print_last(gen):
    j = None
    for j in gen:
        pass
    print(j)

In [58]:
contractive2(x,y) = (0.5*x+3, 0.5*y)

print("Todos las aproximaciones de la primera variante")
gen = first_variant(newton, contractive2, (0.5,.5), 0.0000005)

fix_point, error = None, None
for fix_point, error in gen:
    print("Aproximación: {}, distancia entre términos: {}".format(fix_point, error))
    
print("Todos las aproximaciones de la tercera variante")
gen = third_variant(contractive2, (5,5), 0.8, 10e-9)

fix_point = None, None
for fix_point in gen:
    print("Aproximación: {}, error: {}".format(fix_point, abs(vector((6,0))-fix_point)))

print("")
print("Última aproximación de la segunda variante")
print_last(second_variant(functional_iteration, contractive2, (5,5), fix_point, 10e-9))

Todos las aproximaciones de la primera variante
Aproximación: (-6.00000000000000, 0.000000000000000), distancia entre términos: 0
Todos las aproximaciones de la tercera variante
Iterations: 95
Aproximación: (5.50000000000000, 2.50000000000000), error: 2.54950975679639
Aproximación: (5.75000000000000, 1.25000000000000), error: 1.27475487839820
Aproximación: (5.87500000000000, 0.625000000000000), error: 0.637377439199098
Aproximación: (5.93750000000000, 0.312500000000000), error: 0.318688719599549
Aproximación: (5.96875000000000, 0.156250000000000), error: 0.159344359799775
Aproximación: (5.98437500000000, 0.0781250000000000), error: 0.0796721798998873
Aproximación: (5.99218750000000, 0.0390625000000000), error: 0.0398360899499436
Aproximación: (5.99609375000000, 0.0195312500000000), error: 0.0199180449749718
Aproximación: (5.99804687500000, 0.00976562500000000), error: 0.00995902248748591
Aproximación: (5.99902343750000, 0.00488281250000000), error: 0.00497951124374295
Aproximación: (5.